# InfoSubvenciones API scratchpad
Consultas sin scraping, filtros del cliente y descarga de PDFs/documentos. Modular para ejecutar por bloques.

In [ ]:
import json, urllib.parse, urllib.request, pathlib
import math
try:
    import pandas as pd
    import matplotlib.pyplot as plt
except ImportError as e:
    raise SystemExit('Instala pandas y matplotlib para visualizar: pip install pandas matplotlib')

# Config base
BASE = "https://www.infosubvenciones.es/bdnstrans/api"
OUT_DIR = pathlib.Path('downloads')
RELEVANT_DIR = pathlib.Path('relevant_pdfs')
for folder in (OUT_DIR, RELEVANT_DIR):
    folder.mkdir(exist_ok=True)

# Helpers HTTP
def fetch_json(path: str, params=None):
    qs = urllib.parse.urlencode(params or {}, doseq=True)
    url = f"{BASE}/{path}"
    if qs:
        url = f"{url}?{qs}"
    with urllib.request.urlopen(url) as r:
        return json.load(r)

def fetch_blob(path: str, params=None, target: pathlib.Path=None):
    qs = urllib.parse.urlencode(params or {}, doseq=True)
    url = f"{BASE}/{path}"
    if qs:
        url = f"{url}?{qs}"
    with urllib.request.urlopen(url) as r:
        data = r.read()
    if target:
        target.write_bytes(data)
    return data


## Filtros solicitados por el cliente
- Convocatorias abiertas (`abierto=true`), es decir con plazo vigente.
- Finalidad Cultura (11) y Comercio (14).
- Beneficiarios: PYMES y aut?nomos (`tiposBeneficiario=3,2`).
- Nota: `finalidad` no acepta lista separada por comas, hay que lanzar dos consultas y unir resultados.
- Ajusta `size`/`page` para m?s resultados (aqu? cargamos hasta 100 por tipo para tener algo que visualizar).
- El notebook es modular: ejecuta bloque a bloque seg?n necesites (no depende de ejecuci?n global).

In [ ]:
# Llamadas de b?squeda por filtro (Cultura y Comercio)
f_cultura = {
    'abierto': 'true',  # solo vigentes
    'finalidad': '11',
    'tiposBeneficiario': '3,2',
    'page': 0,
    'size': 100,  # ajusta seg?n necesidad
}
f_comercio = {
    'abierto': 'true',  # solo vigentes
    'finalidad': '14',
    'tiposBeneficiario': '3,2',
    'page': 0,
    'size': 100,  # ajusta seg?n necesidad
}
cultura = fetch_json('convocatorias/busqueda', f_cultura)
comercio = fetch_json('convocatorias/busqueda', f_comercio)
print('Cultura totales:', cultura['totalElements'], 'muestra:', len(cultura['content']))
print('Comercio totales:', comercio['totalElements'], 'muestra:', len(comercio['content']))
pd.DataFrame(cultura['content'])[['numeroConvocatoria','descripcion','fechaRecepcion','nivel1','nivel2']].head()


## Tabla combinada y visualizaci?n r?pida
- Creamos un DataFrame con Cultura+Comercio.
- Vemos campos clave.
- Mostramos un gr?fico de las 10 entidades convocantes m?s frecuentes (`nivel2`).

In [ ]:
# Unir resultados y etiquetar la categor?a
df_cultura = pd.DataFrame(cultura['content'])
df_cultura['categoria'] = 'Cultura'
df_comercio = pd.DataFrame(comercio['content'])
df_comercio['categoria'] = 'Comercio'
df = pd.concat([df_cultura, df_comercio], ignore_index=True)
print('Filas en muestra combinada:', len(df))
display(df[['numeroConvocatoria','categoria','descripcion','fechaRecepcion','nivel1','nivel2']].head(10))
# Conteo de convocantes (nivel2)
top_convocantes = df['nivel2'].value_counts().dropna().head(10)
top_convocantes.plot(kind='barh', figsize=(8,4), title='Top 10 convocantes en la muestra (Cultura+Comercio)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


## Detalle y documentos de una convocatoria de ejemplo
- El detalle se obtiene con `convocatorias?numConv=<numeroConvocatoria>`.
- Los documentos se descargan con `convocatorias/documentos?idDocumento=<idDocumento>`.
- El PDF completo del detalle se descarga con `convocatorias/pdf?id=<idConvocatoria>&vpd=GE`.

In [ ]:
sample = cultura['content'][0] if cultura['content'] else None
if sample:
    num_conv = sample['numeroConvocatoria']
    detalle = fetch_json('convocatorias', {'numConv': num_conv})
    print('numero', num_conv, 'id', detalle.get('id'))
    print('plazo', detalle.get('fechaInicioSolicitud'), '->', detalle.get('fechaFinSolicitud'))
    print('docs', len(detalle.get('documentos', [])))
    if detalle.get('documentos'):
        print('doc ejemplo', detalle['documentos'][0])
else:
    print('No hay resultados de cultura en la muestra')


In [ ]:
# Descarga de todos los documentos de la convocatoria de ejemplo (si existen)
if sample and detalle.get('documentos'):
    for doc in detalle['documentos']:
        target = OUT_DIR / f"{num_conv}_{doc['id']}_{doc['nombreFic']}"
        fetch_blob('convocatorias/documentos', {'idDocumento': doc['id']}, target)
        print('guardado', target)
else:
    print('Sin documentos que descargar en el ejemplo')


In [ ]:
# Descarga del PDF completo del detalle de la convocatoria de ejemplo
if sample:
    pdf_target = OUT_DIR / f"{num_conv}_detalle.pdf"
    fetch_blob('convocatorias/pdf', {'id': detalle.get('id'), 'vpd': 'GE'}, pdf_target)
    print('PDF guardado en', pdf_target)
else:
    print('Sin muestra para descargar PDF')


## Descargar PDFs relevantes (abiertas) en carpeta dedicada
- Recorre resultados abiertos (Cultura y Comercio) y guarda el PDF principal de detalle.
- Ajusta `max_to_download` para limitar volumen.

In [ ]:
max_to_download = 10  # cambia este l?mite si quieres m?s
descargadas = 0
for row in pd.concat([df_cultura, df_comercio]).itertuples():
    if descargadas >= max_to_download:
        break
    num_conv = row.numeroConvocatoria
    detalle = fetch_json('convocatorias', {'numConv': num_conv})
    conv_id = detalle.get('id')
    if not conv_id:
        continue
    pdf_target = RELEVANT_DIR / f"{num_conv}_detalle.pdf"
    fetch_blob('convocatorias/pdf', {'id': conv_id, 'vpd': 'GE'}, pdf_target)
    descargadas += 1
    print(f'[{descargadas}] PDF guardado en {pdf_target}')
print('Total PDFs descargados:', descargadas)
